# Import dependencies

In [ ]:
import os, sys, pandas as pd, matplotlib.pyplot as plt, plotly.offline as py, glob, json

import matplotlib.pyplot as plt, plotly.express as px, webbrowser, json, pandas as pd, os, cv2

from dash import Dash, dcc, html, Input, Output

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import manual_labeler_functions as man_lab_fun

## Analyzes the graphs of the selected video series

In [ ]:
def PLOT_MEASURE_EXPLORER(df):

    port = 5000
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    app = Dash(__name__, external_stylesheets=external_stylesheets)

    def open_browser():
    	webbrowser.open_new("http://localhost:{}".format(port))
          
    app.layout = html.Div([
        html.H4('Measure Explorer'),
        html.H6('Reference Signal'),
        html.I('Setting the frame interval range'),
        html.Br(),
        dcc.Input(id='input-init', type='number', min=df.index.min(), max=df.index.max(), value=df.index.min()),
        dcc.Input(id='input-end', type='number', value=df.index.max()),
        dcc.Graph(id="graph"),
        dcc.Checklist(
            id="checklist",
            options=df.columns,
            value=["m1", "m3"],
            inline=True
        ),
        dcc.Store(
        id='data-output',
        data=[{
            'measures': ["m1", "m3"],
            'init_label': df.index.min(),
            'end_label': df.index.max()
        }]
        ),
        html.Br(),
        html.Details([
        html.Summary('Input Parameters'),
        dcc.Markdown(id='clientside-figure-json')
        ])
    ])
    
    @app.callback(
        Output("graph", "figure"), 
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_line_chart(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[frame_init:frame_end, mask]
        fig = px.line(filtered_df, width=900, height=400, labels={
                     "frame_seq": "<b>Frame number</b>",
                     "value": "<b>Amplitude (pixels)</b>",
                     "variable": "<b>Measures</b>"
                 },)
        data = measures
        fig.update_layout(
            plot_bgcolor='white',
            font_family="Times Nseparate_intervalsew Roman",
            font_size=12,
        )
        fig.update_xaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        fig.update_yaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        return fig

    @app.callback(
        Output('data-output', 'data'),
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_store_data(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[:,mask]
        return [{
            'measures': filtered_df.columns,
            'init_label': frame_init,
            'end_label': frame_end
        }]
    
    
    @app.callback(
        Output('clientside-figure-json', 'children'),
        Input('data-output', 'data')
    )
    def generated_data_json(data):
        return '```\n'+json.dumps(data, indent=2)+'\n```'
   
    if __name__ == '__main__':
        app.run_server(debug=True, port=port)

## Defining folders

In [ ]:
SOURCE_FOLDER = os.path.join('..', 'Dataset', 'REF-Gold-Label')

# Call the basic function to find all VD_MEASURE_L0 files
FOLDER_PATH = os.path.join(SOURCE_FOLDER, '**','*' + ".CSV")
FILE_LOCATION_TREE_DLOCAL = sorted(glob.iglob(FOLDER_PATH, recursive=True ))

# 1 - Generates an index with information about reference series.
INDEX_DT = man_lab_fun.CREATE_LABELED_INDEX(FILE_LOCATION_TREE_DLOCAL)

## Selects the video for analysis and CSV data file

In [ ]:
# Type the video id to label
VIDEO_ID = 1

VD_MEASURE_FILE_NAME = 'VD_MEASURE_L0.CSV'
VIDEO_FILE_PATH = str(INDEX_DT.loc[VIDEO_ID,'path']).split()[2]

# Read CSV data file
VD_MEASURE_DT = man_lab_fun.READ_CSV_FILE(VIDEO_FILE_PATH, VD_MEASURE_FILE_NAME)

# set frames_seq as index
VD_MEASURE_DT_V2 = VD_MEASURE_DT.set_index(pd.Index(VD_MEASURE_DT['frame_seq']))
VD_MEASURE_DT_V2.drop(columns=["frame_seq"], inplace=True)

In [ ]:
PLOT_MEASURE_EXPLORER(VD_MEASURE_DT_V2)

## Loads frames from the selected video for further image analysis 


In [ ]:
BASE_VIDEO_PATH = os.path.join('..', 'Video-Source', 'in_REF-Gold')
VIDEO_EXT = '.mp4'

# Get path from selected video id
video_id = str(INDEX_DT.loc[VIDEO_ID,'link_video']).split()[2] + VIDEO_EXT
path_video_file = os.path.join(BASE_VIDEO_PATH, video_id)

video_frames = man_lab_fun.LOAD_VIDEO_FRAMES(path_video_file)

## Displays frames for image analysis

In [ ]:
start_frame_crop = 0
end_frame_crop = 4
n_frames_per_row = 6
man_lab_fun.DISPLAY_FRAMES(video_frames, start_frame_crop, end_frame_crop, n_frames_per_row)

## Reads or creates the VD_LABELED file if it does not exist.

In [ ]:
labeled_file_exists = int(str(INDEX_DT.loc[VIDEO_ID,'label_file_exist']).split()[2])

if labeled_file_exists == 0:
    print('Creating Label File...')
    VD_MEASURE_DT['label_measures'] = str({})
    VD_LABELED_DT = VD_MEASURE_DT.copy()
else:
    VD_LABEL_FILE_NAME = 'VD_LABELED_L0.CSV'
    print('Reading Label File...') 
    VD_LABELED_DT = man_lab_fun.READ_CSV_FILE(str(INDEX_DT.loc[VIDEO_ID,'path']).split()[2], VD_LABEL_FILE_NAME)

## Label the selected frames

In [ ]:
# Type the Begin and End frames you want to label
init_frame_label = 0
end_frame_label = 38

# Prepare List and Dict
selected_class = 'smile'
label_measure_insert = ['m3', 'm12']

# Just Insert
VD_LABELED_DT = man_lab_fun.UPDATE_LABEL_DF(init_frame_label, end_frame_label, selected_class, label_measure_insert, VD_LABELED_DT)

## Verify the added labels.

In [ ]:
try:
    get_measure = man_lab_fun.GET_MEASURES_FROM_CLASS (VD_LABELED_DT, selected_class)
    print(get_measure)
except:
    print('Class not found')

## Plots a graph marking the start and end of the labels for a class.

In [ ]:
# Select the begin and end frame to be shown
start_frame = 0
end_frame = len(VD_LABELED_DT)

py.init_notebook_mode(connected=True)
plt.rcParams['font.size'] = 10
plt.rcParams["font.family"] = "Times New Roman"

man_lab_fun.PLOT_CLASS_GRAPH(VD_LABELED_DT, VD_MEASURE_DT_V2, selected_class, start_frame, end_frame)

## Save the VD_LABELED file to disk

In [ ]:
VD_LABEL_FILE_NAME = 'VD_LABELED_L0.CSV'
VD_LABELED_DT.to_csv(os.path.join(VIDEO_FILE_PATH, VD_LABEL_FILE_NAME))